In [ ]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime, timedelta
current_mand_symbols = ["US30", "NAS100", "USDJPY", "XAUUSD", "GER30", "JPN225"]
current_mand_timeframes = {
    mt5.TIMEFRAME_M1: 69,
    mt5.TIMEFRAME_M5: 300,
    mt5.TIMEFRAME_M15: 1000,
    mt5.TIMEFRAME_D1: 1200
}


if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

eurcad_rates = mt5.copy_rates_range(
    "US30",
    mt5.TIMEFRAME_D1, 
    datetime.now() - timedelta(1200), 
    datetime.now())

df = pd.DataFrame(eurcad_rates)
df = df.iloc[:, :7]
import pytz

# 先轉成 UTC
df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
df['da'] = df['time'].dt.strftime("%Y-%m-%d")
df['hms'] = df['time'].dt.strftime("%H:%M:%S")
df.sort_values(by=['da', 'hms'], inplace=True)
df = df.drop_duplicates(subset=['da', 'hms'])
df = df.iloc[:, :]
df.columns = ["time", "op", "hi", "lo", "cl", "tick_volume", "spread","da", "hms"]
df['code'] = "US30"
df.head()

,time,op,hi,lo,cl,tick_volume,spread,da,hms,code
0,2022-10-20 00:00:00+00:00,30367.7,30827.7,30258.7,30375.6,156633,0,2022-10-20,00:00:00,US30
1,2022-10-21 00:00:00+00:00,30366.1,31158.6,30068.0,31156.6,226785,0,2022-10-21,00:00:00,US30
2,2022-10-24 00:00:00+00:00,31213.8,31600.2,30882.5,31475.6,254011,0,2022-10-24,00:00:00,US30
3,2022-10-25 00:00:00+00:00,31474.3,31878.7,31308.6,31836.1,203156,0,2022-10-25,00:00:00,US30
4,2022-10-26 00:00:00+00:00,31762.2,32174.8,31708.9,31955.1,208260,0,2022-10-26,00:00:00,US30


In [ ]:
strategy_list = [
    "XAUUSD_rangeBreakout_6amto9am_21pm_revPos_true",
    "USDJPY_rangeReverse_9amto12am_21pm_revPos_true",
    "GER30_rangeBreakout_15pmto1530pm_21pm_revPos_true",
    "GER30_candleReverse_15pmto1515pm_1517pm_oneOrder",
    "JPN225_rangeBreakout_8amto815am_12am_revPos_true",
    "NAS100_trendFollowing",
    "US30_candleReverse_2130pmto2135pm_2140pm_oneOrder",
]

In [1]:
from tqdm import tqdm
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import pytz
import psycopg2

class MT5Inserter:
    def __init__(self, db_params):
        self.db_params = db_params
        if not mt5.initialize():
            raise Exception("MT5 initialize() failed")

    def fetch_data(self, symbol: str, timeframe, days: int) -> pd.DataFrame:
        rates = mt5.copy_rates_range(
            symbol,
            timeframe,
            datetime.now() - timedelta(days=days),
            datetime.now()
        )

        if rates is None or len(rates) == 0:
            print(f"⚠️ No data returned for {symbol}")
            return pd.DataFrame()

        df = pd.DataFrame(rates)
        df = df.iloc[:, :7]  # drop real_volume if exists
        df['time'] = pd.to_datetime(df['time'], unit='s', utc=True)
        df['da'] = df['time'].dt.date
        df['hms'] = df['time'].dt.strftime('%H:%M:%S')
        df.columns = ["time", "op", "hi", "lo", "cl", "tick_volume", "spread", "da", "hms"]
        df['symbol'] = symbol
        df = df.drop_duplicates(subset=['symbol', 'time'])
        return df

    def insert_to_postgres(self, df: pd.DataFrame, table: str):
        if df.empty:
            print("❌ Empty DataFrame, nothing to insert.")
            return

        conn = psycopg2.connect(**self.db_params)
        cur = conn.cursor()

        insert_sql = f"""
        INSERT INTO {table} (
            symbol, time, op, hi, lo, cl, tick_volume, spread, da, hms
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        )
        ON CONFLICT (symbol, time) DO NOTHING;
        """

        data = df[[
            'symbol', 'time', 'op', 'hi', 'lo', 'cl',
            'tick_volume', 'spread', 'da', 'hms'
        ]].values.tolist()

        try:
            cur.executemany(insert_sql, data)
            conn.commit()
        except Exception as e:
            conn.rollback()
            print("❌ Error during insert:", e)
        finally:
            cur.close()
            conn.close()

    def get_symbols(self):
        symbols = mt5.symbols_get()
        return [s.name for s in symbols]

    def shutdown(self):
        mt5.shutdown()


ModuleNotFoundError: No module named 'MetaTrader5'

In [17]:
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "P910317p",
    "host": "localhost",
    "port": 5432
}

inserter = MT5Inserter(db_params)

In [ ]:
current_mand_timeframes = {
    mt5.TIMEFRAME_M1: 69,
    mt5.TIMEFRAME_M5: 300,
    mt5.TIMEFRAME_M15: 1000,
    mt5.TIMEFRAME_D1: 1200
}

tf_tables = {
    mt5.TIMEFRAME_M1: "price_mt5_m1",
    mt5.TIMEFRAME_M5: "price_mt5_m5",
    mt5.TIMEFRAME_M15: "price_mt5_m15",
    mt5.TIMEFRAME_D1: "price_mt5_d1"
}

update_mand_timeframes = {
    mt5.TIMEFRAME_M1: 5,
    mt5.TIMEFRAME_M5: 5,
    mt5.TIMEFRAME_M15: 5,
    mt5.TIMEFRAME_D1: 5
}

In [ ]:
s = inserter.get_symbols()
init = True
if init:
    used_dict = current_mand_timeframes
else:
    used_dict = update_mand_timeframes
for symbol in tqdm(s):
    for tf, count in used_dict.items():
        df = inserter.fetch_data(symbol, tf, count)
        inserter.insert_to_postgres(df, tf_tables[tf])

  0%|          | 0/76 [00:00<?, ?it/s]

✅ Inserted 70353 rows into price_mt5_m1
✅ Inserted 61001 rows into price_mt5_m5


  1%|▏         | 1/76 [00:15<18:57, 15.17s/it]

✅ Inserted 68106 rows into price_mt5_m15
✅ Inserted 854 rows into price_mt5_d1
✅ Inserted 70269 rows into price_mt5_m1
✅ Inserted 61008 rows into price_mt5_m5


  3%|▎         | 2/76 [00:34<21:49, 17.70s/it]

...

✅ Inserted 65167 rows into price_mt5_m15
✅ Inserted 827 rows into price_mt5_d1
✅ Inserted 70475 rows into price_mt5_m1
✅ Inserted 61020 rows into price_mt5_m5


 21%|██        | 16/76 [05:13<19:40, 19.67s/it]

✅ Inserted 68114 rows into price_mt5_m15
✅ Inserted 855 rows into price_mt5_d1
✅ Inserted 69010 rows into price_mt5_m1
✅ Inserted 60249 rows into price_mt5_m5


 22%|██▏       | 17/76 [05:34<19:40, 20.00s/it]

✅ Inserted 67937 rows into price_mt5_m15
✅ Inserted 854 rows into price_mt5_d1
✅ Inserted 70472 rows into price_mt5_m1
✅ Inserted 60997 rows into price_mt5_m5
